# 정부혁신1번가 - 정부혁신 홈페이지
* 정부혁신 제안(목록) > 정부혁신국민포럼 

출처 : https://www.innogov.go.kr/ucms/ogp/sug/list.do?menuNo=300011

```
ㆍ내가 낸 정부혁신 제안이 추천 100표 이상 얻으면, 자동으로 2단계 토론으로 진행됩니다.
ㆍ정책제안이 100표 이상 추천을 받지 못한 경우에도 정부혁신국민포럼 운영위원단 논의를 통해 중요하고 시급한 현안으로 결정된 경우 다음 단계인 토론으로 
   진행될 수 있습니다.
ㆍ제안기간은 90일입니다.
ㆍ욕설 및 비속어를 사용한 제안은 관리자에 의해 삭제 될 수 있습니다.
ㆍ유해하며 폭력적이고 선정적인 내용을 담은 제안은 관리자에 의해 삭제 될 수 있습니다.
ㆍ한번 작성된 제안은 삭제가 불가능합니다.
ㆍ허위 사실이나 타인의 명예를 훼손하는 내용이 포함된 제안은 관리자에 의해 '숨김'처리 또는 삭제될 수 있습니다.
```

## 크롤링 전 확인 하기
* https://www.innogov.go.kr/robots.txt
* 저작권 확인하기

In [1]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러올 수 있습니다.
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용합니다.
from bs4 import BeautifulSoup as bs
# 랜덤숫자를 생성한다.
import random
import time
# 대량 데이터 처리시 진행 상황을 표시합니다.
from tqdm import tqdm, trange
# 정규표현식
import re

In [2]:
# 연-월별 전체 제안 목록을 가져옵니다.
def get_suggestion_list(year_month, pnum):
    # print(f'year_month : {year_month}, pnum: {pnum}')
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&sugMonthTp={year_month}&orderKey=registDtDesc"
    # base_url 페이지를 받아온다.
    response = requests.get( base_url )
    # status_code 가 200 OK 일 때 아래의 코드를 실행한다.
    if response.status_code == 200:
        # 웹페이지의 내용을 가져왔다면 뷰티풀숩을 통해 html 태그를 가져온다.
        html = bs(response.text, 'html.parser')
        # html 태그 내용 중 아래의 selector를 통해 목록에 있는 다른 게시글을 가져오도록 한다.
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        # tags 의 내용이 없다면 suggestion_list 를 반환한다.
        if not tags :
            return(suggestion_list)
        else :
            # tags 에서 다른 게시글을 가져와 suggestion_list 에 담아준다.
            for tag in tags:
                suggestion_list.append(tag['href'])
        pnum += 1
        # 페이지번호를 하나씩 더해서 재귀호출을 합니다.
        get_suggestion_list(year_month, pnum)
    else:
        return(suggestion_list)

In [3]:
duration = []
# 게시물 목록을 가져오기 위해 파라메터에 넘겨줄 값을 리스트로 만듭니다.
# 2018년 7월부터 12월까지를 리스트로 생성합니다.
# 좋은 방법은 아니니 이 코드를 보고 개선해 보시면 좋을 것 같아요.
for month in range(7, 13):
    if month < 10:
        month = f'0{month}'
    duration.append(f'2018{month}')

for month in range(1, 5):
    if month < 10:
        month = f'0{month}'
    duration.append(f'2019{month}')
    
print(duration)

['201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']


In [4]:
# 목록의 내용만 크롤링 해옵니다.
# 목록의 링크 값을 가져와 리스트에 담아줍니다.
suggestion_list = []
pnum = 1

for year_month in tqdm(duration):
    get_suggestion_list(year_month, pnum)
    suggestion_list    

100%|██████████| 10/10 [00:10<00:00,  1.07s/it]


In [5]:
print(len(suggestion_list))

suggestion_list[:3]

133


['/ucms/ogp/sug/view.do?menuNo=300011&sgId=1&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=28&pnum=1',
 '/ucms/ogp/sug/view.do?menuNo=300011&sgId=26&pnum=1']

In [6]:
# id 추출 미리 해보기
u = 'view.do?menuNo=300011&sgId=150&pnum=1'
u.split('=')[2].split('&')[0]

'150'

In [7]:
# 목록 리스트에 있는 url를 넘겨주면 내용을 크롤링 해옵니다.
def get_suggestion_content(url):
    article = []
    base_url = f"https://www.innogov.go.kr/{url}"
    response = requests.get( base_url )
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        # 제목을 가져옵니다.
        title = html.select(
            '#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        # dd 태그에 있는 내용을 가져와서 카테고리, 시작일, 마감일, 제안인 정보를 추출합니다.
        desc = html.select(
            '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = desc[0].get_text(strip=True)
        content = html.select(
            '#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)
        # 추천인원을 가져옵니다.
        vote = html.select('#counter')[0].get_text(strip=True)
        # 해당 게시물의 Id 값을 가져옵니다.
        sgId = url.split('=')[2].split('&')[0]
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
        #print(article)
        
        time.sleep(random.randint(1,2))
        return article

In [8]:
data = []
for i, suggestion in tqdm(enumerate(suggestion_list)):
    # suggestion_list 에 있는 각 게시물에 접근해서 카테고리, 시작일, 마감일, 제안인, 제목, 내용 등을 가져옵니다.
    s = get_suggestion_content(suggestion)
    data.append(s)

133it [03:52,  1.75s/it]


In [9]:
column_names = ["sgId", "title", "category", "content", "start", "end", "vote", "author"]
column_names

['sgId', 'title', 'category', 'content', 'start', 'end', 'vote', 'author']

In [10]:
# data

In [11]:
data = pd.DataFrame(data, columns=column_names)
data.head()

sgId                                  title category  \
0    1  어린이집 차량내 어린이 질식사고 등에 대한 상세한 대책이 필요하다.       안전   
1   28                              성희롱 없는 세상       안전   
2   26                   행정안전부소속의 온라인 일자리뱅크운영     상생협력   
3   25                       농어업,농어촌 특별위원회 설치     국민참여   
4   24                   혁신적인 귀농창업교육의 패러다임 제시      공동체   

                                             content       start         end  \
0  최근 동두천 어린이집 차량내 어린이 질식사고가 발생하여, 이에 대한 여러가지 대책을...  2018.07.24  2018.08.23   
1  현재 우리 사회에서 익숙해진 대형 범죄가 있습니다.바로 몰래카메라입니다.현재 공공건...  2018.08.31  2018.11.28   
2  정부각부처에서는 제각각 일자리창출에 혼신의 힘을 쏟고 있습니다.문제는 제각각하다보니...  2018.08.28  2018.11.25   
3  청년실업률이 높아지고 일자리 찾기는 점점 어려워지면서농업과 농촌으로 눈을 돌리는 분...  2018.08.19  2018.11.16   
4  농촌의 생력을 불어넣을 대안으로 귀농귀촌지원정책은 정부,지자체,농협등의 관련기관들이...  2018.08.16  2018.11.13   

  vote author  
0   14     헌비  
1    3    장대연  
2    4   생태조성  
3   15   생태조성  
4    5   생태조성

In [12]:
data["category"].value_counts()

기타       24
일반행정     22
보건복지     19
안전       16
인권/평등    12
상생협력     10
국민참여      8
환경        7
반부패청렴     6
사회통합      6
공동체       3
Name: category, dtype: int64

In [13]:
data.to_csv("data/suggestion.csv", index=False)

In [14]:
pd.read_csv("data/suggestion.csv").head()

sgId                                  title category  \
0     1  어린이집 차량내 어린이 질식사고 등에 대한 상세한 대책이 필요하다.       안전   
1    28                              성희롱 없는 세상       안전   
2    26                   행정안전부소속의 온라인 일자리뱅크운영     상생협력   
3    25                       농어업,농어촌 특별위원회 설치     국민참여   
4    24                   혁신적인 귀농창업교육의 패러다임 제시      공동체   

                                             content       start         end  \
0  최근 동두천 어린이집 차량내 어린이 질식사고가 발생하여, 이에 대한 여러가지 대책을...  2018.07.24  2018.08.23   
1  현재 우리 사회에서 익숙해진 대형 범죄가 있습니다.바로 몰래카메라입니다.현재 공공건...  2018.08.31  2018.11.28   
2  정부각부처에서는 제각각 일자리창출에 혼신의 힘을 쏟고 있습니다.문제는 제각각하다보니...  2018.08.28  2018.11.25   
3  청년실업률이 높아지고 일자리 찾기는 점점 어려워지면서농업과 농촌으로 눈을 돌리는 분...  2018.08.19  2018.11.16   
4  농촌의 생력을 불어넣을 대안으로 귀농귀촌지원정책은 정부,지자체,농협등의 관련기관들이...  2018.08.16  2018.11.13   

   vote author  
0    14     헌비  
1     3    장대연  
2     4   생태조성  
3    15   생태조성  
4     5   생태조성

In [15]:
data.shape

(133, 8)